In [26]:
import os
import numpy as np
import cv2

# ダミーデータの作成
data = []
labels = []

# 画像ファイルのパス
ALMOND_path = "/Users/sakurako/Lecture/BohPJ/ALMOND1.jpeg"
Ghana_path = "/Users/sakurako/Lecture/BohPJ/Ghana1.jpeg"

# ALMONDの画像処理
img = cv2.imread(ALMOND_path)
if img is not None:
    img = cv2.resize(img, (64, 64))  # 画像のサイズを統一する
    data.append(img)
    labels.append(0)  # ALMONDのラベルを0とする
else:
    print(f"Failed to load image: {ALMOND_path}")

# Ghanaの画像処理
img = cv2.imread(Ghana_path)
if img is not None:
    img = cv2.resize(img, (64, 64))
    data.append(img)
    labels.append(1)  # Ghanaのラベルを1とする
else:
    print(f"Failed to load image: {Ghana_path}")

# データのシャッフル
if data and labels:
    data = np.array(data)
    labels = np.array(labels)
    shuffle_indices = np.random.permutation(len(data))
    data = data[shuffle_indices]
    labels = labels[shuffle_indices]

    data = data /255.0

    print("Data shape:", data.shape)
    print("Labels shape:", labels.shape)
else:
    print("No data to shuffle.")


Data shape: (2, 64, 64, 3)
Labels shape: (2,)


In [29]:
import cv2
import numpy as np

# 畳み込み層の実装
def conv2d(input_data, filters, bias, stride=1, padding=1):
    input_data_padded = np.pad(input_data, ((0, 0), (padding, padding), (padding, padding), (0, 0)), mode='constant')
    filter_height, filter_width, input_channels, num_filters = filters.shape
    batch_size, input_height, input_width, _ = input_data.shape
    
    output_height = (input_height - filter_height + 2 * padding) // stride + 1
    output_width = (input_width - filter_width + 2 * padding) // stride + 1
    output = np.zeros((batch_size, output_height, output_width, num_filters))
    
    for b in range(batch_size):
        for h in range(0, input_height - filter_height + 1, stride):
            for w in range(0, input_width - filter_width + 1, stride):
                for f in range(num_filters):
                    output[b, h//stride, w//stride, f] = np.sum(input_data_padded[b, h:h+filter_height, w:w+filter_width, :] * filters[:, :, :, f]) + bias[f]
    return output

# Max Pooling 層の実装
def max_pooling(input_data, pool_size=2, stride=2):
    batch_size, input_height, input_width, input_channels = input_data.shape
    output_height = (input_height - pool_size) // stride + 1
    output_width = (input_width - pool_size) // stride + 1
    output = np.zeros((batch_size, output_height, output_width, input_channels))
    
    for b in range(batch_size):
        for h in range(0, input_height - pool_size + 1, stride):
            for w in range(0, input_width - pool_size + 1, stride):
                for c in range(input_channels):
                    output[b, h//stride, w//stride, c] = np.max(input_data[b, h:h+pool_size, w:w+pool_size, c])
    return output

# Flatten 層の実装
def flatten(input_data):
    return input_data.reshape(input_data.shape[0], -1)

# Dense 層の実装
def dense(input_data, weights, bias):
    return np.dot(input_data, weights) + bias

# 活性化関数
def relu(x):
    return np.maximum(0, x)

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

# データの仮定
data = np.random.randn(32, 64, 64, 3)  # ダミーデータ
labels = np.random.randint(2, size=(32, 1))  # ダミーラベル

# 畳み込み層のパラメータ
filters = np.random.randn(3, 3, 3, 32)
bias_conv = np.random.randn(32)

# 全結合層のパラメータ
weights_dense1 = np.random.randn(32*32*32, 128)
bias_dense1 = np.random.randn(128)
weights_dense2 = np.random.randn(128, 1)
bias_dense2 = np.random.randn(1)

# 順伝播
conv_output = relu(conv2d(data, filters, bias_conv))
pool_output = max_pooling(conv_output)
flat_output = flatten(pool_output)
dense_output1 = relu(dense(flat_output, weights_dense1, bias_dense1))
output = sigmoid(dense(dense_output1, weights_dense2, bias_dense2))

# 損失関数の計算（バイナリクロスエントロピー）
loss = -np.mean(labels * np.log(output) + (1 - labels) * np.log(1 - output))
print(f"Loss: {loss}")

# 簡単な精度計算
predicted_labels = (output > 0.5).astype(int)
accuracy = np.mean(predicted_labels == labels)
print(f"Accuracy: {accuracy}")

# 予測関数
def predict_image(image_path):
    img = cv2.imread(image_path)
    img = cv2.resize(img, (64, 64))
    img = np.expand_dims(img, axis=0)
    
    # 順伝播
    conv_output = relu(conv2d(img, filters, bias_conv))
    pool_output = max_pooling(conv_output)
    flat_output = flatten(pool_output)
    dense_output1 = relu(dense(flat_output, weights_dense1, bias_dense1))
    prediction = sigmoid(dense(dense_output1, weights_dense2, bias_dense2))
    
    if prediction[0][0] > 0.5:
        return "GHANA アレルギー物質は入っていません"
    else:
        return "ALMOND アーモンドが入っています"

# 予測の実行
image_path = "/Users/sakurako/Lecture/BohPJ/Ghana2.jpeg"
prediction = predict_image(image_path)
print("予測結果:", prediction)


/var/folders/g5/4n67_nfs6sj21cxpk5sm98ww0000gn/T/ipykernel_64668/775573231.py:48: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-x))
/var/folders/g5/4n67_nfs6sj21cxpk5sm98ww0000gn/T/ipykernel_64668/775573231.py:72: RuntimeWarning: divide by zero encountered in log
  loss = -np.mean(labels * np.log(output) + (1 - labels) * np.log(1 - output))
/var/folders/g5/4n67_nfs6sj21cxpk5sm98ww0000gn/T/ipykernel_64668/775573231.py:72: RuntimeWarning: invalid value encountered in multiply
  loss = -np.mean(labels * np.log(output) + (1 - labels) * np.log(1 - output))


Loss: nan
Accuracy: 0.40625
予測結果: GHANA アレルギー物質は入っていません
